# 🦞 CoLobster

Run your own **OpenClaw** AI gateway on a free Google Colab instance — 3 clicks and you're live.

---
## Step 1 — Add Your API Key(s)

OpenClaw needs at least **one** LLM provider key. You set them up once in Colab’s secure Secrets storage — they persist across sessions and never appear in the notebook.

### First time? Here’s how:

1. Open the **🔑 Secrets** panel — click the key icon in the left sidebar
2. Click **"+ Add new secret"**
3. Type the **Name** exactly as shown below, paste your key as the **Value**
4. Toggle **"Notebook access"** ON

Add any of these (you only need one):

| Name (copy exactly) | Provider | Get a key | Free tier? |
|---|---|---|---|
| `GEMINI_API_KEY` | Google Gemini | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) | ✅ Yes |
| `OPENROUTER_API_KEY` | OpenRouter | [openrouter.ai/keys](https://openrouter.ai/keys) | ✅ Some models |
| `ANTHROPIC_API_KEY` | Anthropic (Claude) | [console.anthropic.com](https://console.anthropic.com/settings/keys) | ❌ Pay-as-you-go |
| `OPENAI_API_KEY` | OpenAI (GPT) | [platform.openai.com/api-keys](https://platform.openai.com/api-keys) | ❌ Pay-as-you-go |

> 💡 **Easiest free start:** Get a Gemini key — it’s free, takes 30 seconds, and works great.

### Already done this before?

Your secrets are saved in Colab. Just make sure **Notebook access** is toggled ON for this notebook, then move on to Step 2.

---
## Step 2 — Install & Configure

This cell installs Node.js + OpenClaw and detects your API keys. Takes a few minutes on first run.

In [ ]:
#@title ▶️ Click to install & configure
import subprocess, os, secrets, sys, time, json
from IPython.display import display, HTML

# ============================================================
# Phase 1: Install Node.js 22 + OpenClaw
# ============================================================
display(HTML('<b style="color:#333">⏳ Installing Node.js & OpenClaw...</b>'))

# Check Node.js
node_ok = False
try:
    ver = subprocess.run(['node', '-v'], capture_output=True, text=True)
    major = int(ver.stdout.strip().lstrip('v').split('.')[0])
    node_ok = major >= 22
except Exception:
    pass

if not node_ok:
    subprocess.run('curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash -',
                   shell=True, capture_output=True)
    subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'], capture_output=True)

# Check OpenClaw
oc = subprocess.run(['which', 'openclaw'], capture_output=True, text=True)
if not oc.stdout.strip():
    r = subprocess.run(['npm', 'install', '-g', 'openclaw@latest'], capture_output=True, text=True)
    if r.returncode != 0:
        print(r.stderr[-1000:])
        raise RuntimeError('npm install failed')

node_v = subprocess.run(['node', '-v'], capture_output=True, text=True).stdout.strip()
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()

display(HTML(f'''
<div style="background:#e8f5e9; color:#1b5e20; border-radius:6px; padding:8px 14px; margin:4px 0; font-size:13px;">
  ✅ Node.js {node_v} &nbsp;·&nbsp; OpenClaw at <code style="color:#2e7d32">{oc_bin}</code>
</div>
'''))

# ============================================================
# Phase 2: Detect API keys from Colab Secrets
# ============================================================
try:
    from google.colab import userdata
    _has_colab = True
except ImportError:
    _has_colab = False

KEY_NAMES = ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GEMINI_API_KEY', 'OPENROUTER_API_KEY']

found = {}
for name in KEY_NAMES:
    val = None
    if _has_colab:
        try:
            val = userdata.get(name)
        except Exception:
            pass
    if not val:
        val = os.environ.get(name)
    if val and val.strip():
        found[name] = val.strip()
        os.environ[name] = val.strip()

if not found:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:16px; margin:12px 0;">
      <b>⚠️ No API keys found.</b><br><br>
      Go back to <b>Step 1</b> and add at least one key in the 🔑 Secrets panel.<br>
      Make sure <b>"Notebook access"</b> is toggled ON, then re-run this cell.
    </div>
    '''))
else:
    # Show what we found
    provider_names = {
        'ANTHROPIC_API_KEY': 'Anthropic',
        'OPENAI_API_KEY': 'OpenAI',
        'GEMINI_API_KEY': 'Gemini',
        'OPENROUTER_API_KEY': 'OpenRouter',
    }
    detected = [provider_names[k] for k in found]

    # ============================================================
    # Phase 3: Prepare state dir & gateway token
    # ============================================================
    # API keys stay in environment variables only — no .env file on disk.
    # The gateway process inherits them via os.environ.copy() in Step 3.
    STATE_DIR = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
    os.makedirs(STATE_DIR, exist_ok=True)
    os.environ['OPENCLAW_STATE_DIR'] = STATE_DIR

    token = secrets.token_hex(32)
    os.environ['OPENCLAW_GATEWAY_TOKEN'] = token

    display(HTML(f'''
    <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:12px 16px; margin:8px 0;">
      ✅ <b>Ready!</b> Keys detected: {', '.join(detected)}<br>
      <span style="font-size:13px;">OpenClaw will automatically pick the best model for your keys. Run <b>Step 3</b> to start.</span>
    </div>
    '''))

---
### Choose a Model (optional)

OpenClaw auto-selects a model from your available keys, but defaults to **Claude Opus 4.6** — which is powerful but expensive.

Run the cell below to pick a different model. If you skip this, OpenClaw uses its default.

> 💡 **Cheapest start:** Gemini 3 Flash is free and fast. OpenRouter also gives access to models from DeepSeek, Moonshot (Kimi), xAI (Grok) and more.

In [ ]:
#@title ▶️ Choose a model
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# Available models per provider key (curated list — updated Feb 2026)
PROVIDER_MODELS = {
    'GEMINI_API_KEY': [
        ('Gemini 3 Flash Preview  —  free, fast', 'google/gemini-3-flash-preview'),
        ('Gemini 3.1 Pro Preview  —  free, powerful', 'google/gemini-3.1-pro-preview'),
    ],
    'ANTHROPIC_API_KEY': [
        ('Claude Sonnet 4.6  —  balanced', 'anthropic/claude-sonnet-4-6'),
        ('Claude Haiku 4.5  —  fast, cheap', 'anthropic/claude-haiku-4-5'),
        ('Claude Opus 4.6  —  most capable, expensive', 'anthropic/claude-opus-4-6'),
    ],
    'OPENAI_API_KEY': [
        ('GPT-5 Nano  —  fast, cheap', 'openai/gpt-5-nano'),
        ('GPT-5.2  —  balanced', 'openai/gpt-5.2'),
        ('GPT-5.2 Pro  —  most capable, expensive', 'openai/gpt-5.2-pro'),
    ],
    'OPENROUTER_API_KEY': [
        ('Gemini 3 Flash (via OR)  —  free', 'openrouter/google/gemini-3-flash-preview'),
        ('DeepSeek V3.2 (via OR)  —  fast, cheap', 'openrouter/deepseek/deepseek-v3.2'),
        ('Kimi K2.5 (via OR)  —  powerful', 'openrouter/moonshotai/kimi-k2.5'),
        ('Grok 4.1 Fast (via OR)', 'openrouter/x-ai/grok-4.1-fast'),
        ('Qwen 3.5 Plus (via OR)', 'openrouter/qwen/qwen3.5-plus-02-15'),
        ('MiniMax M2.5 (via OR)', 'openrouter/minimax/minimax-m2.5'),
    ],
}

# Build options from detected keys
options = []
for key_name in ['GEMINI_API_KEY', 'ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'OPENROUTER_API_KEY']:
    if os.environ.get(key_name):
        options.extend(PROVIDER_MODELS[key_name])

if not options:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:14px;">
      <b>⚠️</b> No API keys detected. Run <b>Step 2</b> first, then come back here.
    </div>
    '''))
else:
    # Pre-select the first option (cheapest/free model for the first detected provider)
    dropdown = widgets.Dropdown(
        options=options,
        value=options[0][1],
        description='',
        layout=widgets.Layout(width='450px'),
        style={'description_width': '0px'},
    )

    def on_change(change):
        os.environ['OPENCLAW_SELECTED_MODEL'] = change['new']

    dropdown.observe(on_change, names='value')

    # Set initial selection
    os.environ['OPENCLAW_SELECTED_MODEL'] = dropdown.value

    display(HTML('<b style="font-size:14px; color:#333;">Select a model:</b>'))
    display(dropdown)
    display(HTML(f'''
    <div style="background:#e8f5e9; color:#1b5e20; border-radius:6px; padding:8px 14px; margin:8px 0; font-size:13px;">
      ✅ Selected: <code style="color:#2e7d32">{dropdown.value}</code> — this will be used when you run Step 3.
      <br><span style="color:#555;">Change the dropdown above to pick a different model. You can also change it later via the dashboard.</span>
    </div>
    '''))

---
### Optional: Google Drive Persistence

By default, OpenClaw state (sessions, config) is stored locally and **lost when this Colab runtime shuts down** (after ~12 hours of inactivity, or when you disconnect).

If you want your conversations and settings to survive between sessions, run this cell **before** starting the gateway. It mounts Google Drive and stores OpenClaw state there.

> Skip this if you're just trying things out — you can always come back and enable it later (you'll need to re-run Steps 2 and 3).

In [ ]:
#@title ▶️ Enable Drive storage (optional)
from google.colab import drive
import os, shutil
from IPython.display import display, HTML

drive.mount('/content/drive')

DRIVE = '/content/drive/MyDrive/openclaw/state'
LOCAL = '/content/openclaw_state'
os.makedirs(DRIVE, exist_ok=True)

# Copy any existing local state to Drive
if os.path.exists(LOCAL):
    for item in os.listdir(LOCAL):
        s, d = os.path.join(LOCAL, item), os.path.join(DRIVE, item)
        if os.path.isfile(s): shutil.copy2(s, d)
        elif os.path.isdir(s) and not os.path.exists(d): shutil.copytree(s, d)

os.environ['OPENCLAW_STATE_DIR'] = DRIVE
link = '/content/.openclaw'
if os.path.islink(link): os.unlink(link)
os.symlink(DRIVE, link)

display(HTML('''
<div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:14px;">
  ✅ <b>Drive persistence enabled.</b> State will be saved to Google Drive.<br>
  <span style="font-size:13px;">Now run <b>Step 3</b> to start OpenClaw.</span>
</div>
'''))


---
## Step 3 — Start OpenClaw

In [ ]:
#@title ▶️ Click to launch
import subprocess, os, time, json, re, urllib.request, urllib.error
from IPython.display import display, HTML

STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')

# Preflight checks
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
if not oc_bin:
    display(HTML('<div style="background:#f8d7da; color:#721c24; padding:12px; border-radius:8px;">❌ OpenClaw not found. Run <b>Step 2</b> first.</div>'))
elif not os.environ.get('OPENCLAW_GATEWAY_TOKEN'):
    display(HTML('<div style="background:#f8d7da; color:#721c24; padding:12px; border-radius:8px;">❌ No keys configured. Run <b>Step 2</b> first.</div>'))
else:
    # Write openclaw.json — auth + enable HTTP chat API + Control UI dashboard
    # reload.mode=hot: don't force-restart when the file watcher detects config changes
    # controlUi.basePath: serve dashboard at /ui/ (default is root /)
    # controlUi.allowInsecureAuth: let the dashboard use token-only auth without
    #   device pairing — required for ngrok/tunnel access. Without this, the
    #   Control UI sends a device identity that needs pairing approval, but you
    #   can't approve it because the dashboard itself is what needs pairing.
    # trustedProxies: ngrok connects from 127.0.0.1 — without this, the gateway
    #   sees proxy headers from an untrusted address and rejects the connection
    config = {
        'gateway': {
            'auth': {
                'token': os.environ['OPENCLAW_GATEWAY_TOKEN']
            },
            'http': {
                'endpoints': {
                    'chatCompletions': {
                        'enabled': True
                    }
                }
            },
            'controlUi': {
                'enabled': True,
                'basePath': '/ui',
                'allowInsecureAuth': True
            },
            'reload': {
                'mode': 'hot'
            },
            'trustedProxies': ['127.0.0.1']
        },
        # Restrict agent tools: block shell/process execution so the agent
        # cannot run commands like printenv, cat /proc/self/environ, etc.
        # This prevents prompt-injection attacks that trick the agent into
        # leaking API keys or other secrets from the environment.
        'tools': {
            'deny': ['exec', 'bash', 'process']
        }
    }

    # Apply model selection (from the "Choose a Model" cell, or skip for auto-detect)
    # agents.defaults.model must be an object { primary: "provider/model-id" }, not a plain string
    selected_model = os.environ.get('OPENCLAW_SELECTED_MODEL')
    if selected_model:
        config['agents'] = {
            'defaults': {
                'model': {
                    'primary': selected_model
                }
            }
        }

    config_path = os.path.join(STATE, 'openclaw.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    # Pre-create workspace dir (where MEMORY.md lives)
    os.makedirs(os.path.join(STATE, 'workspace'), exist_ok=True)

    # Environment
    env = os.environ.copy()
    env['OPENCLAW_STATE_DIR'] = STATE
    env['HOME'] = '/content'
    env['NODE_ENV'] = 'production'
    env['OPENCLAW_SKIP_CHANNELS'] = '1'
    env['OPENCLAW_SKIP_GMAIL_WATCHER'] = '1'
    env['OPENCLAW_NO_RESPAWN'] = '1'  # in-process restart instead of process exit (no systemd on Colab)

    # Symlink ~/.openclaw -> state dir
    # Create symlinks at BOTH /content/.openclaw (for the gateway process, which
    # runs with HOME=/content) AND /root/.openclaw (for CLI commands run from the
    # Colab shell, where HOME=/root). Without the /root symlink, `openclaw gateway
    # status` reports "config missing" even though the gateway process found it.
    for link in ['/content/.openclaw', '/root/.openclaw']:
        if os.path.islink(link):
            os.unlink(link)
        if not os.path.exists(link):
            os.symlink(STATE, link)

    # Stop previous instance
    subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
    time.sleep(2)

    def _wait_for_gateway_ready(max_wait=45):
        """Wait until local API responds with any non-5xx HTTP status."""
        token = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')
        headers = {'Authorization': f'Bearer {token}'} if token else {}
        deadline = time.time() + max_wait
        while time.time() < deadline:
            req = urllib.request.Request('http://127.0.0.1:18789/v1/models', headers=headers)
            try:
                with urllib.request.urlopen(req, timeout=2) as resp:
                    if resp.status < 500:
                        return True
            except urllib.error.HTTPError as e:
                if e.code < 500:
                    return True
            except Exception:
                pass
            time.sleep(1)
        return False

    def _launch_gateway(max_attempts=2):
        last_proc = None
        for attempt in range(1, max_attempts + 1):
            if attempt > 1:
                subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
                time.sleep(2)

            with open('/content/openclaw_gateway.log', 'w') as log:
                last_proc = subprocess.Popen(
                    [oc_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
                    env=env, stdout=log, stderr=subprocess.STDOUT, preexec_fn=os.setsid,
                )

            if _wait_for_gateway_ready(max_wait=45):
                return last_proc

            if last_proc.poll() is not None:
                continue

        return last_proc

    display(HTML('<b style="color:#333">⏳ Starting OpenClaw...</b>'))
    proc = _launch_gateway(max_attempts=2)

    if proc and proc.poll() is None and _wait_for_gateway_ready(max_wait=5):
        token_preview = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')[:16]

        # Read the log to find what model OpenClaw picked (strip ANSI color codes)
        model_line = ''
        try:
            with open('/content/openclaw_gateway.log') as f:
                for line in f:
                    if 'agent model:' in line:
                        model_line = re.sub(r'\x1b\[[0-9;]*m', '', line.split('agent model:')[1]).strip()
                        break
        except Exception:
            pass
        model_display = model_line or selected_model or 'auto-detected'

        display(HTML(f'''
        <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:16px; margin:8px 0;">
          <h3 style="margin:0 0 10px 0; color:#155724;">✅ OpenClaw is running!</h3>
          <table style="border:none; border-collapse:collapse; font-size:14px; color:#1a1a1a;">
            <tr><td style="padding:3px 12px 3px 0"><b>Model</b></td><td><code style="color:#2e7d32">{model_display}</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Port</b></td><td><code style="color:#2e7d32">18789</code> <span style="color:#777; font-size:12px;">(internal — use Chat UI below or set up a tunnel for external access)</span></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Logs</b></td><td><code style="color:#2e7d32">/content/openclaw_gateway.log</code></td></tr>
          </table>
          <p style="margin:10px 0 0 0; font-size:13px; color:#333;">
            Scroll down to <b>"What now?"</b> to see your options.
          </p>
        </div>
        '''))
    else:
        with open('/content/openclaw_gateway.log') as f:
            err = f.read()[-3000:]
        display(HTML(f'''
        <div style="background:#f8d7da; color:#721c24; border:1px solid #dc3545; border-radius:8px; padding:16px;">
          <b>❌ Gateway failed to start after retry.</b>
          <pre style="margin-top:8px; font-size:12px; max-height:300px; overflow:auto; color:#721c24;">{err}</pre>
        </div>
        '''))


---
## What now?

OpenClaw is running, so you can stay in a single non-technical interface from here.

| Option | What it does | Best for |
|---|---|---|
| **🕹️ CoLobster Control Center** | Gradio app with chat, gateway controls, logs, and quick setup links | **Recommended** daily use |
| **🖥️ OpenClaw Dashboard** | Native OpenClaw UI at `localhost:18789/ui/` | Channel linking (WhatsApp/Telegram), advanced settings |
| **🌐 ngrok tunnel** | Makes your Colab runtime reachable from outside | Linking phone apps, external webhooks, remote access |

> ✅ **Recommended flow:** run the **Control Center** cell below and manage everything from that interface.
>
> ⚠️ **WhatsApp linking tip:** use the Dashboard channel setup screen (local or tunneled URL). Avoid terminal-only pairing where possible.

---


---
### 🕹️ CoLobster Control Center (Gradio)

A single interface with:
- 💬 Chat against the running OpenClaw gateway
- 🔄 Restart / stop / status checks without terminal commands
- 📜 Live gateway log tail for easier troubleshooting
- 🔗 Quick links for dashboard + channel setup (including WhatsApp/Telegram)


In [ ]:
#@title ▶️ Launch CoLobster Control Center
!pip install gradio requests -q

import os
import subprocess
import requests
import gradio as gr

URL = 'http://127.0.0.1:18789'
TOK = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '').strip()
LOG_FILE = '/content/openclaw_gateway.log'

_SECRET_NAMES = [
    'ANTHROPIC_API_KEY',
    'OPENAI_API_KEY',
    'GEMINI_API_KEY',
    'OPENROUTER_API_KEY',
    'OPENCLAW_GATEWAY_TOKEN',
    'NGROK_AUTH_TOKEN',
]
_SECRETS = {}
for _name in _SECRET_NAMES:
    _val = os.environ.get(_name, '').strip()
    if _val and len(_val) >= 8:
        _SECRETS[_name] = _val

def _redact_secrets(text):
    for name, val in _SECRETS.items():
        if val in text:
            text = text.replace(val, f'[REDACTED {name}]')
    return text

def _gateway_status_text():
    r = subprocess.run(['pgrep', '-f', 'openclaw.*gateway'], capture_output=True, text=True)
    if r.stdout.strip():
        pid = r.stdout.strip().split('\\n')[0]
        return f'✅ Gateway running (PID {pid})'
    return '❌ Gateway is not running. Run restart from this tab.'

def _tail_logs(lines=120):
    if not os.path.exists(LOG_FILE):
        return 'No log file yet.'
    try:
        out = subprocess.run(['tail', f'-{lines}', LOG_FILE], capture_output=True, text=True)
        return out.stdout or '(log file is empty)'
    except Exception as e:
        return f'Unable to read logs: {e}'

def _run_action(action):
    if action == 'stop':
        subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
        return '⏹️ Stop command sent.'
    if action == 'restart':
        subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
        oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
        if not oc_bin:
            return '❌ OpenClaw binary not found. Re-run install cell.'
        env = os.environ.copy()
        env['OPENCLAW_STATE_DIR'] = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
        env['HOME'] = '/content'
        env['NODE_ENV'] = 'production'
        env['OPENCLAW_SKIP_CHANNELS'] = '1'
        env['OPENCLAW_SKIP_GMAIL_WATCHER'] = '1'
        env['OPENCLAW_NO_RESPAWN'] = '1'
        with open(LOG_FILE, 'w') as log:
            subprocess.Popen(
                [oc_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
                env=env,
                stdout=log,
                stderr=subprocess.STDOUT,
                preexec_fn=os.setsid,
            )
        return '🔄 Restart command sent. Wait ~5-15s, then refresh status.'
    return 'Unknown action.'

def _refresh_panel():
    ngrok_url = os.environ.get('OPENCLAW_PUBLIC_URL', '').strip()
    public_hint = ngrok_url if ngrok_url else 'Not set. Run the ngrok cell if you need remote linking.'
    dashboard_local = 'http://127.0.0.1:18789/ui/'
    setup_md = (
        f"### Quick links\n"
        f"- Dashboard (local): [{dashboard_local}]({dashboard_local})\n"
        f"- Public URL: {public_hint}\n"
        f"- WhatsApp guide: https://docs.openclaw.ai/channels/whatsapp\n"
        f"- Telegram guide: https://docs.openclaw.ai/channels/telegram\n\n"
        f"**Tip:** link channels from the dashboard UI instead of terminal commands whenever possible."
    )
    return _gateway_status_text(), _tail_logs(), setup_md

def send_message(message, history):
    history = history or []
    if not message.strip():
        return '', history

    for name, val in _SECRETS.items():
        if val in message:
            history.append({'role': 'assistant', 'content': f'⚠️ Message contains a secret ({name}) and was blocked.'})
            return '', history

    messages = [{'role': item['role'], 'content': item['content']} for item in history]
    messages.append({'role': 'user', 'content': message})

    try:
        r = requests.post(
            f'{URL}/v1/chat/completions',
            headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {TOK}'},
            json={'model': 'openclaw', 'messages': messages, 'stream': False},
            timeout=120,
        )
        if r.status_code == 200:
            data = r.json()
            choices = data.get('choices', [])
            reply = choices[0].get('message', {}).get('content', str(data)) if choices else str(data)
        else:
            reply = f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.ConnectionError:
        reply = '❌ Gateway not reachable. Go to Gateway tab and press Restart, then Refresh.'
    except requests.exceptions.Timeout:
        reply = '⏳ Request timed out. Try a shorter prompt or faster model.'
    except Exception as e:
        reply = f'Error: {e}'

    history.append({'role': 'user', 'content': message})
    history.append({'role': 'assistant', 'content': _redact_secrets(reply)})
    return '', history

with gr.Blocks(title='CoLobster Control Center') as demo:
    gr.Markdown('## 🦞 CoLobster Control Center')

    with gr.Tab('💬 Chat'):
        chatbot = gr.Chatbot(type='messages', height=420)
        msg = gr.Textbox(label='Message OpenClaw', placeholder='Ask anything…')
        with gr.Row():
            send_btn = gr.Button('Send', variant='primary')
            clear_btn = gr.Button('Clear chat')

    with gr.Tab('🛠️ Gateway'):
        status = gr.Markdown()
        with gr.Row():
            refresh_btn = gr.Button('Refresh status', variant='primary')
            restart_btn = gr.Button('Restart gateway')
            stop_btn = gr.Button('Stop gateway')
        action_result = gr.Markdown()
        logs = gr.Textbox(label='Recent logs', lines=18)

    with gr.Tab('🔗 Channels & Setup'):
        setup_links = gr.Markdown()

    send_btn.click(send_message, [msg, chatbot], [msg, chatbot])
    msg.submit(send_message, [msg, chatbot], [msg, chatbot])
    clear_btn.click(lambda: [], None, chatbot)

    refresh_btn.click(_refresh_panel, None, [status, logs, setup_links])
    restart_btn.click(lambda: _run_action('restart'), None, action_result).then(
        _refresh_panel, None, [status, logs, setup_links]
    )
    stop_btn.click(lambda: _run_action('stop'), None, action_result).then(
        _refresh_panel, None, [status, logs, setup_links]
    )

    demo.load(_refresh_panel, None, [status, logs, setup_links])

demo.launch(share=False, debug=False)



---
### 🌐 Public Tunnel (ngrok)

Google Colab VMs have **no public IP address** — external traffic simply cannot reach the gateway. A tunnel solves this by creating a public URL that routes into this VM.

> ℹ️ Running this cell will **automatically restart the gateway** so it picks up the tunnel. This is normal — like mounting Google Drive, adding a tunnel requires a restart.

#### Without a tunnel (default)

| Feature | Works? |
|---|---|
| **💬 Gradio Chat UI** (in this notebook) | ✅ Yes — runs in the same VM |
| **🖥️ Dashboard** at `localhost:18789/ui/` (this Colab tab only) | ✅ Yes — same VM |
| Access from your phone or another device | ❌ No |
| OpenAI-compatible API from external apps | ❌ No |
| Webhook channels (Slack HTTP, Telegram webhook) | ❌ No |

#### With a tunnel

Everything above **plus**:

| Feature | Works? |
|---|---|
| **🖥️ Dashboard** from any device (phone, laptop) | ✅ Yes |
| **OpenAI-compatible API** (`/v1/chat/completions`) from external apps | ✅ Yes |
| **Slack** (HTTP webhook mode) | ✅ Yes |
| **Telegram** (webhook mode) | ✅ Yes |
| **WebSocket** connections (CLI/SDK clients, real-time dashboard updates) | ✅ Yes |

#### Channels that work WITHOUT a tunnel

Some messaging channels use **outbound** connections (the gateway connects to them, not the other way around):

- **Discord** — socket mode (outbound WebSocket)
- **Telegram** — polling mode (outbound HTTP, no webhook needed)
- **Slack** — socket mode (outbound WebSocket, no HTTP webhook needed)

These work behind Colab's NAT without any tunnel. Configure them via `openclaw.json` or the dashboard.

#### Setup

1. Get a free auth token at [ngrok.com](https://ngrok.com)
2. Add `NGROK_AUTH_TOKEN` to the 🔑 Secrets panel
3. Run the cell below

> ⚠️ **ngrok free tier** gives a random URL that changes every time you restart the tunnel. For a stable URL, consider [Cloudflare Tunnel](https://developers.cloudflare.com/cloudflare-one/connections/connect-networks/) (`cloudflared`) as a free alternative.

In [ ]:
#@title ▶️ Open ngrok tunnel
import os, subprocess, re
from urllib.parse import quote as urlquote
from IPython.display import display, HTML

tok = None
try:
    from google.colab import userdata
    tok = userdata.get('NGROK_AUTH_TOKEN')
except Exception:
    tok = os.environ.get('NGROK_AUTH_TOKEN')

if not tok:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:14px;">
      <b>⚠️</b> Add <code style="color:#856404">NGROK_AUTH_TOKEN</code> to the 🔑 Secrets panel, then re-run.
    </div>
    '''))
else:
    # ── 1. Start ngrok tunnel ────────────────────────────────
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)
    from pyngrok import ngrok, conf
    conf.get_default().auth_token = tok.strip()
    tunnel = ngrok.connect(18789, 'http')
    public_url = tunnel.public_url

    gateway_token = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')
    dashboard_url = f'{public_url}/ui/#token={urlquote(gateway_token)}' if gateway_token else f'{public_url}/ui/'
    api_url = f'{public_url}/v1/chat/completions'

    # ── 2. Write the public URL into MEMORY.md ───────────────
    workspace_dir = os.path.join(os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state'), 'workspace')
    os.makedirs(workspace_dir, exist_ok=True)
    memory_path = os.path.join(workspace_dir, 'MEMORY.md')

    existing = ''
    try:
        with open(memory_path) as f:
            existing = f.read()
    except FileNotFoundError:
        pass

    existing = re.sub(r'## Public Gateway URL\n.*?(?=\n## |\Z)', '', existing, flags=re.DOTALL).strip()

    url_section = f"""## Public Gateway URL

The gateway's public URL is: {public_url}
This ngrok tunnel routes external traffic into the Colab VM.
Use this URL for webhook registrations, sharing with external apps, or accessing the dashboard from other devices.
The dashboard is at: {public_url}/ui/
The OpenAI-compatible chat API is at: {api_url}
"""

    with open(memory_path, 'w') as f:
        content = f"{existing}\n\n{url_section}".strip() + '\n' if existing else url_section
        f.write(content)

    # ── 3. Check gateway status (no forced restart) ─────────
    # ngrok does not require a gateway restart; restarting here can cause
    # unnecessary downtime and false negatives. We only report status.
    gateway_running = bool(subprocess.run(['pgrep', '-f', 'openclaw.*gateway'], capture_output=True, text=True).stdout.strip())

    if not gateway_running:
        display(HTML('''
        <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:10px;">
          ⚠️ Gateway is not running. Run <b>Step 3</b> to start it — the tunnel is ready and waiting.
        </div>
        '''))

    gateway_status_row = (
        '<tr><td style="padding:3px 12px 3px 0"><b>🟢 Gateway</b></td><td style="color:#2e7d32;">Running</td></tr>'
        if gateway_running else
        '<tr><td style="padding:3px 12px 3px 0"><b>🟢 Gateway</b></td><td style="color:#b26a00;">Not running yet (start Step 3)</td></tr>'
    )

    # ── 4. Show results ──────────────────────────────────────
    display(HTML(f'''
    <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:14px;">
      ✅ <b>Tunnel active!</b><br><br>
      <table style="border:none; border-collapse:collapse; font-size:14px; color:#1a1a1a;">
        <tr><td style="padding:3px 12px 3px 0"><b>🖥️ Dashboard</b></td><td><a href="{dashboard_url}" target="_blank" style="color:#2e7d32">{public_url}/ui/</a> <span style="color:#777; font-size:12px;">(token auto-applied)</span></td></tr>
        <tr><td style="padding:3px 12px 3px 0"><b>🔌 API</b></td><td><code style="color:#2e7d32">{api_url}</code></td></tr>
        <tr><td style="padding:3px 12px 3px 0"><b>🌐 Base URL</b></td><td><code style="color:#2e7d32">{public_url}</code></td></tr>
        {gateway_status_row}
      </table>
      <p style="margin:10px 0 0 0; font-size:12px; color:#555;">
        ngrok is live. If gateway status shows not running, start/restart it from <b>Step 3</b>.<br>
        The agent also knows this URL (written to MEMORY.md).
      </p>
    </div>
    '''))

---
### 🛠️ Utilities

In [ ]:
#@title 📜 View logs
!tail -50 /content/openclaw_gateway.log 2>/dev/null || echo 'No log file yet.'

In [ ]:
#@title 🟢 Check status
import subprocess
from IPython.display import display, HTML
r = subprocess.run(['pgrep', '-f', 'openclaw.*gateway'], capture_output=True, text=True)
if r.stdout.strip():
    pid = r.stdout.strip().split('\n')[0]
    display(HTML(f'<div style="background:#d4edda; color:#155724; border-radius:6px; padding:8px 14px; font-size:13px;">✅ <b>Running</b> (PID {pid})</div>'))
else:
    display(HTML('<div style="background:#f8d7da; color:#721c24; border-radius:6px; padding:8px 14px; font-size:13px;">❌ <b>Not running.</b> Re-run Step 3 to restart.</div>'))

In [ ]:
#@title ⏹️ Stop gateway
!pkill -f 'openclaw.*gateway' && echo '✅ Stopped.' || echo 'Not running.'